In [125]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [126]:
dataspam = np.loadtxt("spambase/spambase.data", delimiter=",")
features_name = np.genfromtxt("spambase/spambase_variables.csv", delimiter=",", dtype="str")
features_name = features_name [:,0]
X = dataspam [: ,0:57]
y = dataspam [:,-1]

In [127]:
print(f'Il y a {len(X[y==0])} exemple pour le classe 0 et {len(X[y==1])} pour la classe 1')

Il y a 2788 exemple pour le classe 0 et 1813 pour la classe 1


In [128]:
len(X[y==1])

1813

# 2. Séparez les données en un sous-ensemble d’apprentissage, un sous-ensemble de validation et un sous-ensemble de test. Utilisez un tiers des données pour chaque sous-ensemble.

In [129]:
X,X_test,y,Y_test=train_test_split(X,y,test_size=0.33,stratify=y)
X_train,X_valid,Y_train,Y_valid=train_test_split(X,y,test_size=0.5,stratify=y)

In [130]:
len(X_valid)

1541

# 3. Normalisez les données en utilisant la méthode StandardScaler, comme vu dans le TP3

In [131]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)

# 4. En utilisant les ensembles d’apprentissage et de validation, déterminez la valeur optimale du paramètres C pour un SVM linéaire. Vous pouvez tester des valeurs dans l’intervalle [10−3, 102].

In [ ]:
errer=[]
C=np.logspace(1,3,20)
for c in C:
   clf=SVC(C=c,kernel='linear')
   clf.fit(X_train,Y_train)
   pred=clf.predict(X_valid)
   errer.append(mean_absolute_error(pred,Y_valid))

In [ ]:
plt.plot(C,errer)
plt.xscale('log')

In [134]:
 argminn=np.array(errer).argmin()

In [135]:
C[argminn]

np.float64(1.4563484775012443)